# Lec 18. Word2Vec with CNN - Sentiment Analysis

In [1]:
import pandas as pd
import numpy as np
import random

from konlpy.tag import Okt
from gensim.models import word2vec

import torch
import torch.nn as nn
import torch.utils.data as data_utils

from sklearn.model_selection import train_test_split

C:\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


## Load Data

In [2]:
filename = "data/score_발열.xlsx"
sheet_name = "Sheet1"
data = pd.read_excel(filename, sheet_name = sheet_name, header = 0)

csv_data = [item.replace("#", "").strip() for item in data['Review']]
csv_label = data['Score']

In [3]:
csv_data[:5]

['발열히 심한거 같은데 여름이라 그런가?..',
 '발열이좀 심한거 같아서 걱정이에요',
 '발열이심하더라구요',
 '발열이너무심한게 제일큰 단점인것 같고 그외에 불편한점은',
 '발열이...정말...심합니다']

## Word2Vec Trasformation

In [4]:
twitter = Okt()
size = 500

doc = []

for sentence in csv_data :
    results= []
    tokens = twitter.pos(sentence, norm=True, stem=True)
        
    for token in tokens:
        if not token[1] in ["Josa", "Eomi", "Punctuation"]:
            results.append(token[0])
    doc.append(results)

In [5]:
doc[:5]

[['발열', '히', '심하다', '같다', '여름', '그', '런가'],
 ['발열', '이', '좀', '심하다', '같다', '걱정'],
 ['발열', '심하다'],
 ['발열', '이', '너', '무심하다', '제일', '크다', '단점', '것', '같다', '그', '외', '불편하다', '점'],
 ['발열', '정말', '심하다']]

* sentences (iterable of iterables, optional) – The sentences iterable can be simply a list of lists of tokens, but for larger corpora, consider an iterable that streams the sentences directly from disk/network. See BrownCorpus, Text8Corpus or LineSentence in word2vec module for such examples. See also the tutorial on data streaming in Python. If you don’t supply sentences, the model is left uninitialized – use if you plan to initialize it in some other way.
* size (int, optional) – Dimensionality of the word vectors.
* window (int, optional) – Maximum distance between the current and predicted word within a sentence.
* min_count (int, optional) – Ignores all words with total frequency lower than this.
* workers (int, optional) – Use these many worker threads to train the model (=faster training with multicore machines).
* sg ({0, 1}, optional) – Training algorithm: 1 for skip-gram; otherwise CBOW.
* hs ({0, 1}, optional) – If 1, hierarchical softmax will be used for model training. If 0, and negative is non-zero, negative sampling will be used.
* negative (int, optional) – If > 0, negative sampling will be used, the int for negative specifies how many “noise words” should be drawn (usually between 5-20). If set to 0, no negative sampling is used.

In [6]:
model = word2vec.Word2Vec(doc, size=size, window=2, hs=0, min_count=3, sg=0)

w2v = dict(zip(model.wv.index2word, model.wv.vectors))

del model

In [7]:
w2v.keys()

dict_keys(['발열', '없다', '있다', '하다', '소음', '좋다', '심하다', '같다', '않다', '거의', '좀', '적다', '자다', '것', '잡다', '문제', '조금', '되다', '정도', '생각', '만족하다', '괜찮다', '못', '노트북', '사용', '이', '느끼다', '잘', '너무', '부분', '이다', '팬', '성능', '쿨러', '정말', '돌아가다', '제품', '별로', '걱정', '크다', '배터리', '수', '많이', '더', '전혀', '게임', '보다', '속도', '도', '아직', '매우', '아주', '가볍다', '소리', '안', '쓸다', '어쩔', '느껴지다', '다', '때문', '적', '나다', '그렇다', '키', '편이', '관리', '아니다', '신경', '크게', '조용하다', '약간', '때', '많다', '하', '및', '네', '보이다', '만족', '그', '쓰다', '상당하다', '들다', '없이', '거', '꽤', '보드', '오래', '쿨링', '무게', '아쉽다', '디자인', '제어', '점', '심해', '높다', '빠르다', '양호', '또한', '대', '모두', '빼다', '돌리다', '모르다', '발생', '다른', '맘', '감다', '잡히다', '되어다', '확실하다', '요', '진짜', '들', '삼성', '시간', '해보다', '사양', '가격', '감', '편', '모델', '안나', '가다', '비', '개선', '단점', '사', '느낌', '하지만', '제', '뜨겁다', '펜', '하나', '시', '듯', '사은', '써다', '수준', '최고', '만족스럽다', '은', '구매', '습', '메탈', '약하다', '상태', '충전', '판', '안되다', '굉장하다', '훨씬', '품', '작업', '화면', '발', '열량', '그렇게', '장시간', '생기다', '지다', '중', '무엇', '걸리다', '받침', '구

In [8]:
# Document to 3-dim Matrix with Word2Vector & Get Max Length of Sentence
# 길이가 가장 큰 문장의 길이에 맞춰 0 추가

doc2vec = []
max_length = 0

for sentence in doc :
    temp = []
    length = 0
    
    for word in sentence :
        if word in w2v.keys() :
            temp.append(w2v[word])
            length += 1
            
    doc2vec.append(temp)
    
    if max_length <= length :
        max_length = length

In [9]:
# Fill zeros for fitting size
for sentence in doc2vec :
    
    length = len(sentence)
    
    while length < max_length :
        sentence.append(np.zeros(size))
        length += 1

doc2vec = np.array(doc2vec)

In [10]:
doc2vec.shape

(1211, 12, 500)

In [11]:
data = doc2vec
label = csv_label.values

## Train/Test Spilt

In [12]:
train_data, test_data, train_label, test_label = train_test_split(data, label)

print(len(train_data))
print(len(test_data))

908
303


In [13]:
x = torch.from_numpy(train_data).type(torch.FloatTensor)
y = torch.from_numpy(train_label).type(torch.LongTensor)

In [14]:
y.size() , x.size()

(torch.Size([908]), torch.Size([908, 12, 500]))

In [15]:
x = x.view(-1, 1, 12, 500)

In [16]:
train_data = data_utils.TensorDataset(x, y)

batch_size = 10

train_loader = torch.utils.data.DataLoader(dataset=train_data,
                                           batch_size=batch_size,
                                           shuffle=True,
                                           num_workers=1,
                                           drop_last = True)

In [17]:
text, label = iter(train_loader).next()
text.shape

torch.Size([10, 1, 12, 500])

## Define Model for Sentiment Analysis

In [18]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        self.layer = nn.Sequential(
            nn.Conv2d(1,16,3),   # 1*12*500 -> 16*10*498
            nn.BatchNorm2d(16),
            nn.ReLU(),
            
            nn.Conv2d(16,32,3),  # 16*10*498 -> 32*8*496
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2,2),   # 32*8*496 -> 32*4*248
            
            nn.Conv2d(32,64,3),  # 32*4*248 -> 64*2*246
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2,2)    # 64*2*246 -> 64*1*123
        )

        self.fc_layer = nn.Sequential(
            nn.Linear(64*1*123,100),
            nn.ReLU(),
            nn.Linear(100,3)
        )       
        
    def forward(self,x):
        out = self.layer(x)
        out = out.view(-1,64*1*123)
        out = self.fc_layer(out)

        return out

model = CNN().cuda()

In [19]:
loss = torch.nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [20]:
num_epochs = 30

In [21]:
for epoch in range(num_epochs):
    avg_cost = 0
    total_batch = len(train_data) // batch_size
    
    for i, (batch_text, batch_labels) in enumerate(train_loader):
        
        X = batch_text.cuda()
        Y = batch_labels.cuda()
        
        pred = model(X)
        cost = loss(pred, Y)
        
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        
        if (i+1) == total_batch:
            print('Epoch [%d/%d], lter [%d/%d] Loss: %.4f'%(epoch+1, num_epochs, i+1, total_batch, cost.item()))
    
print("Learning Finished!")

Epoch [1/30], lter [90/90] Loss: 0.6230
Epoch [2/30], lter [90/90] Loss: 0.9019
Epoch [3/30], lter [90/90] Loss: 0.7881
Epoch [4/30], lter [90/90] Loss: 0.5576
Epoch [5/30], lter [90/90] Loss: 0.8675
Epoch [6/30], lter [90/90] Loss: 0.8376
Epoch [7/30], lter [90/90] Loss: 0.8033
Epoch [8/30], lter [90/90] Loss: 1.0412
Epoch [9/30], lter [90/90] Loss: 0.6446
Epoch [10/30], lter [90/90] Loss: 0.7519
Epoch [11/30], lter [90/90] Loss: 0.5286
Epoch [12/30], lter [90/90] Loss: 0.8123
Epoch [13/30], lter [90/90] Loss: 0.6668
Epoch [14/30], lter [90/90] Loss: 0.8470
Epoch [15/30], lter [90/90] Loss: 0.5784
Epoch [16/30], lter [90/90] Loss: 0.4752
Epoch [17/30], lter [90/90] Loss: 0.6593
Epoch [18/30], lter [90/90] Loss: 1.2282
Epoch [19/30], lter [90/90] Loss: 0.6718
Epoch [20/30], lter [90/90] Loss: 0.4601
Epoch [21/30], lter [90/90] Loss: 0.7973
Epoch [22/30], lter [90/90] Loss: 1.4645
Epoch [23/30], lter [90/90] Loss: 0.7047
Epoch [24/30], lter [90/90] Loss: 0.9515
Epoch [25/30], lter [90/9

## 18.5 Test Model

In [22]:
x_test = torch.from_numpy(test_data).type(torch.FloatTensor)
y_test = torch.from_numpy(test_label).type(torch.LongTensor)

test_data = data_utils.TensorDataset(x, y)

test_loader = torch.utils.data.DataLoader(dataset=test_data,
                                          batch_size=1,
                                          shuffle=True)

In [23]:
model.eval()

correct = 0
total = 0

for text, labels in test_loader:
    
    text = text.cuda()
    outputs = model(text)
    
    _, pred = torch.max(outputs.data, 1)
    total += 1
    correct += (pred == labels.cuda()).sum()
    
correct = correct.cpu().numpy()    
print('correct :', correct)
print('total   :', total)
print('Accuracy of test images: %f' % (100 * correct / total))

correct : 761
total   : 908
Accuracy of test images: 83.810573
